   # Lab 2 - Template

The template is given as starting point to help you get up to speed with loading file and understanding data.  
Use the template and answer each question in the appropriate cell.


Naming: Meeran_Siddiqui_mxs220072_Lab_2.ipynb

## Dataset description

The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## The function `describe_more` is data exploration utility

It is up to you if you want to use the function. If you find bugs, please let me know but try to fix them yourself.  
The function most definetely doesn't work with every input

In [2]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:200].to_dict())
                
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

## Load data

We assume data is located in the folder `data` where we run our notebook

In [69]:
data = pd.read_csv("C:\\Users\\ASUS\\Downloads\\SBA_loans_small(1).csv")
data.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,CHGOFF,"$26,000.00","$23,400.00"


In [70]:
data.shape

(448583, 20)

In [71]:
desc_df = describe_more(data)

In [72]:
pd.set_option('display.width', 200)
desc_df.head(n=200)

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,24757,object,1,30,"{'LOS ANGELES': 5751, 'HOUSTON': 5102, 'NEW YO..."
1,State,51,object,2,3,"{'CA': 65183, 'TX': 35161, 'NY': 28716, 'FL': ..."
2,Zip,28185,int64,1,5,"{90015: 492, 10001: 455, 93401: 385, 90010: 37..."
3,Bank,5156,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 43347, 'WELLS F..."
4,BankState,54,object,2,3,"{'CA': 58768, 'NC': 39709, 'IL': 32982, 'OH': ..."
5,NAICS,1290,int64,1,6,"{0: 100776, 722110: 13888, 722211: 9713, 81111..."
6,Term,392,int64,1,3,"{84: 114519, 60: 44888, 240: 43020, 120: 38908..."
7,NoEmp,466,int64,1,4,"{1: 76992, 2: 68936, 3: 45269, 4: 36503, 5: 30..."
8,NewExist,3,float64,3,3,"{1.0: 321541, 2.0: 126457, 0.0: 520}"
9,CreateJob,196,int64,1,4,"{0: 313743, 1: 31439, 2: 29216, 3: 14422, 4: 1..."


## Question 1

How many of the each target label are there? Target label is `MIS_Status` column. 

In [35]:
target_counts = data["MIS_Status"].value_counts()
print(target_counts)

0    369804
1     78779
Name: MIS_Status, dtype: int64


## Question 2

Encode target variable `MIS_Status` into `0` and `1`, when value `P I F` is mapped to `0` and `CHGOFF` is mapped to `1`.   

Replace values in the original column

In [76]:
df=data
df['MIS_Status'].replace({'P I F': 0, 'CHGOFF': 1}, inplace=True)
df['MIS_Status']=df['MIS_Status'].astype(int)
df.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00"


## Question 3

Think about how to encode `ZIP` column? 

- Should we leave it as is? 
- Should we encode it using one-hot-encoding?
- Should we encode it using target encoder?

#### Question 3a

Copy original Pandas dataframe into new dataframe.  
Encode `ZIP` column using `from sklearn.preprocessing import OneHotEncoder`  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Check new dataframe. Is it good idea to use one-hot-encoder on `ZIP` column? 

In [87]:
from sklearn.preprocessing import OneHotEncoder
df1=data.copy()

encoder = OneHotEncoder()
encoded_df = pd.DataFrame(encoder.fit_transform(df1[['Zip']]).toarray())

df1 = df1.join(encoder_df)

#I think it is not a good idea to use OneHotEncoder because it is causing Memoryerror.


MemoryError: Unable to allocate 94.2 GiB for an array with shape (448583, 28185) and data type float64

### Question 3b (corrected on 1/26)

Encode ZIP column using target **(mean)** encoding technique.
At this point `MIS_Status` column contains only 0 and 1.
Replace `ZIP` values with the mean of the response column (`MIS_Status`) for each value in a `ZIP` column. 

For example, if for ZIP=11222 there are total of 150 records. Out of which 80 records have label `0` and 70 records have label `1`, populate new column value for the record with replace with value:  
(0x80 + 1x70)/(80+70) = 0.467

The approach is not entirely  correct because of overfitting, but will give you general idea of one potential target encoding technique.  

Please see video on feature engineering topic posted in Module-2 for the explanation why frequency encoding is not ideal without modifications.

In [88]:
target_encoding= df.groupby('Zip')['MIS_Status'].mean()

# replace the original ZIP values with the target encoded values
df['ZIP_encoded'] = df['Zip'].map(target_encoding)

In [32]:
df
# The resulting dataframe will contain a new column "ZIP_encoded" with the target encoded values
print(df)


                    City State    Zip                            Bank  \
0                BOULDER    CO  80302     WELLS FARGO BANK NATL ASSOC   
1                SEATTLE    WA  98101                      HANMI BANK   
2                PHOENIX    AZ  85023          CAPITAL ONE NATL ASSOC   
3                HOUSTON    TX  77074  JPMORGAN CHASE BANK NATL ASSOC   
4             LAS CRUCES    NM  88001      BANK OF AMERICA NATL ASSOC   
...                  ...   ...    ...                             ...   
448578  HUNTINGTON BEACH    CA  92649     WELLS FARGO BANK NATL ASSOC   
448579             DOVER    NH   3820        CITIZENS BANK NATL ASSOC   
448580           LUBBOCK    TX  79423              PLAINSCAPITAL BANK   
448581          SULLIVAN    IL  61951   FIRST MID-ILLINOIS BK & TRUST   
448582          SYRACUSE    NY  13202       THE UPSTATE NATIONAL BANK   

       BankState   NAICS  Term  NoEmp  NewExist  CreateJob  RetainedJob  \
0             SD  327122    39      8       1.0 

## Question 4 - replace the column with categorical (corrected on 1/26)

Encode `LowDoc` column using `from sklearn import preprocessing.LabelEncoder`. Replace original column. 

In [33]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

df["LowDoc"] = label_encoder.fit_transform(df["LowDoc"])
df

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,ZIP_encoded
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,4,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00",0.113636
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,4,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00",0.080882
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,4,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00",0.153846
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,4,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00",0.247191
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,4,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00",0.049645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1.0,0,0,1,0,N,4,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00",0.125000
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2.0,0,1,0,1,N,4,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00",0.081967
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2.0,1,1,52007,1,N,4,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00",0.080808
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1.0,0,0,1,0,N,7,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00",0.111111


## Question 5

Fill all columns with missing values with `Unknown` or equivalent value.

```
# filling with Unknown class
df_clean = df.fillna("Unknown")
df_clean
```

In [ ]:
for col in data.columns:
    if data[col].isna().any() == True:
        print(col)

In [41]:
df_clean = df.fillna("Unknown")
df_clean

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,ZIP_encoded
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,4,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00",0.113636
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,4,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00",0.080882
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,4,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00",0.153846
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,4,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00",0.247191
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,4,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00",0.049645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1.0,0,0,1,0,N,4,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00",0.125000
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2.0,0,1,0,1,N,4,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00",0.081967
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2.0,1,1,52007,1,N,4,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00",0.080808
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1.0,0,0,1,0,N,7,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00",0.111111


## Question 6

Load same data into H2O-3 dataframe.  
Describe imported dataframe.

In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.351-b10, mixed mode, sharing)
  Starting server from C:\Users\ASUS\Virtualenv\ml-spring-2023\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ASUS\AppData\Local\Temp\tmpp4ws_0sw
  JVM stdout: C:\Users\ASUS\AppData\Local\Temp\tmpp4ws_0sw\h2o_ASUS_started_from_python.out
  JVM stderr: C:\Users\ASUS\AppData\Local\Temp\tmpp4ws_0sw\h2o_ASUS_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_ASUS_bxwz7j
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
df_h = h2o.import_file("E:\\machine_learning\\SBA_loans_small(1).csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
df_h.describe()

Rows:448583
Cols:20

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,int,enum,enum
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,1.0,,
mean,,,53835.004286832074,,,398521.68325817067,110.85133854827312,11.351830096102614,1.2807847176701923,8.801037935008685,11.131210946469219,2761.0083440522717,0.7576658054362286,,,,0.0013375600230060324,1.0,,
maxs,,,99999.0,,,928120.0,569.0,9999.0,2.0,8800.0,8800.0,92006.0,2.0,,,,600.0,1.0,,
sigma,,,31206.493657954878,,,263210.820607375,78.92372956114484,74.55017951410956,0.4519555984078703,243.34264128531348,243.5617653492284,12778.87936703734,0.6462952056079755,,,,0.8958437440779611,0.0,,
zeros,,,141,,,100776,430,3345,520,313743,219626,104260,161211,,,,448577,0,,
missing,17,9,0,761,766,0,0,0,65,0,0,0,0,130956,2040,0,5,369804,0,0
0,BOULDER,CO,80302.0,WELLS FARGO BANK NATL ASSOC,SD,327122.0,39.0,8.0,1.0,0.0,8.0,1.0,1.0,Y,N,"$60,639.00",0.0,1.0,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101.0,HANMI BANK,CA,722211.0,15.0,2.0,1.0,0.0,2.0,1.0,1.0,,N,"$66,000.00",0.0,1.0,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023.0,CAPITAL ONE NATL ASSOC,VA,323119.0,62.0,15.0,1.0,0.0,0.0,1.0,1.0,N,N,"$50,000.00",0.0,1.0,"$50,000.00","$25,000.00"


In [5]:
dfh_na = df_h.fillna()

In [6]:
df_h["Zip"]

Zip
80302
98101
85023
77074
88001
44708
83404
48180
78502
85238


In [3]:
df_h.shape

(448583, 20)

## Question 7

Use `fillna` method to fill missing values for the following columns:
- City
- State
- Bank
- BankState
- NewExist
- RevLineCr
- LowDoc

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/fillnas.html

In [ ]:
num_col= df_h.columns_by_type(coltype="numeric")
str_col= df_h.columns_by_type(coltype="string")
cat_col= df_h.columns_by_type(coltype="categorical")

print(num_col,str_col,cat_col)
df_h[['Zip','NAICS','Term','NoEmp','NewExist','CreateJob','RetainedJob','FranchiseCode','UrbanRural','BalanceGross']] = df_h[['Zip','NAICS','Term','NoEmp','NewExist','CreateJob','RetainedJob','FranchiseCode','UrbanRural','BalanceGross']].fillna(method = "forward", axis = 0, maxlen = 10)
df_h[['City','State','Bank','BankState','RevLineCr','LowDoc']] = df_h[['City','State','Bank','BankState','RevLineCr','LowDoc']].fillna(method = "forward", axis = 0, maxlen = 100)
df_h
for col in df_h.columns:
    if df_h[col].isna().any()==True:
        print(col)


[2.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 16.0, 17.0] [] [0.0, 1.0, 3.0, 4.0, 13.0, 14.0, 15.0, 18.0, 19.0]


In [ ]:
df_h.describe()

## Question 8

Perform target encoding on the following columns:
- ZIP
- City
- State

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html


In [5]:
from h2o.estimators import H2OTargetEncoderEstimator
df_h[["City", "State", "Zip"]] =df_h[["City", "State", "Zip"]].asfactor()
targetDf = df_h[["City", "State", "Zip"]]
targetDf
target_encoder = H2OTargetEncoderEstimator()
df_h['MIS_Status'] = df_h['MIS_Status'].asfactor()
response='MIS_Status'

# Split the dataset into train and test
train, test = df_h.split_frame(ratios = [.8], seed = 1234)

# Choose which columns to encode
encoded_columns = ["Zip", "City", "State"]

# For k_fold strategy we need to provide fold column
fold_column = "kfold_column"
train[fold_column] = train.kfold_column(n_folds=5, seed=1234)

# Train a TE model
targetDf = H2OTargetEncoderEstimator(fold_column=fold_column,
                                       data_leakage_handling="k_fold",
                                       blending=True,
                                       inflection_point=3,
                                       smoothing=10,
                                       noise=0.15,
                                       seed=1234)

targetDf.train(x=encoded_columns,
                 y=response,
                 training_frame=train)
train_te = targetDf.transform(frame=train, as_training=True)
test_te = targetDf.transform(frame=test, noise=0)
train_te
test_te


H2OResponseError: ModelBuilderErrorV3  (water.exceptions.H2OModelBuilderIllegalArgumentException):
    timestamp = 1675665382673
    error_url = '/3/ModelBuilders/targetencoder'
    msg = 'Illegal argument(s) for TargetEncoder model: TargetEncoder_model_python_1675665208488_1.  Details: ERRR on field: _response: Response cannot be constant.'
    dev_msg = 'Illegal argument(s) for TargetEncoder model: TargetEncoder_model_python_1675665208488_1.  Details: ERRR on field: _response: Response cannot be constant.'
    http_status = 412
    values = {'messages': [{'_log_level': 5, '_field_name': '_fold_assignment', '_message': 'Fold assignment is ignored when a fold column is specified.'}, {'_log_level': 5, '_field_name': '_nfolds', '_message': 'nfolds is ignored when a fold column is specified.'}, {'_log_level': 5, '_field_name': '_keep_cross_validation_models', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_keep_cross_validation_predictions', '_message': 'Only